In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client('intsql-2025')

# Construct a reference to the "world_bank_intl_education" dataset
dataset_ref = client.dataset("world_bank_intl_education", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "international_education" table
table_ref = dataset_ref.table("international_education")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "international_education" table
client.list_rows(table, max_results=5).to_dataframe()

/Users/oktavianu/miniforge3/envs/data-analysis/lib/python3.11/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/var/folders/7m/y8yqbwcj4_s9pbq64p2cdkw00000gn/T/ipykernel_5176/3950152268.py:19: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  client.list_rows(table, max_results=5).to_dataframe()


,country_name,country_code,indicator_name,indicator_code,value,year
0,Chad,TCD,"Enrolment in lower secondary education, both s...",UIS.E.2,321921.0,2012
1,Chad,TCD,"Enrolment in upper secondary education, both s...",UIS.E.3,68809.0,2006
2,Chad,TCD,"Enrolment in upper secondary education, both s...",UIS.E.3,30551.0,1999
3,Chad,TCD,"Enrolment in upper secondary education, both s...",UIS.E.3,79784.0,2007
4,Chad,TCD,"Repeaters in primary education, all grades, bo...",UIS.R.1,282699.0,2006


#### Exercises
The value in the `indicator_code` column describes what type of data is shown in a given row.

One interesting indicator code is `SE.XPD.TOTL.GD.ZS`, which corresponds to "Government expenditure on education as % of GDP (%)".

**1) Government expenditure on education¶**
Which countries spend the largest fraction of GDP on education?

To answer this question, consider only the rows in the dataset corresponding to indicator code SE.XPD.TOTL.GD.ZS, and write a query that returns the average value in the value column for each country in the dataset between the years 2010-2017 (including 2010 and 2017 in the average).

Requirements:
- The results should have the country name rather than the country code. One row for each country.
- The aggregate function for average is AVG(). Use the name avg_ed_spending_pct for the column created by this aggregation.
- Order the results so the countries that spend the largest fraction of GDP on education show up first.

Example of previous query related to this:
```sql
query = """
        SELECT COUNT(consecutive_number) AS num_accidents, 
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """
```

In [3]:
country_spend_pct_query = """
                        SELECT country_name, AVG(value) AS avg_ed_spending_pct
                        FROM `bigquery-public-data.world_bank_intl_education.international_education`
                        WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' AND year BETWEEN 2010 AND 2017
                        GROUP BY country_name
                        ORDER BY avg_ed_spending_pct DESC
                        """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
country_spend_pct_query_job = client.query(country_spend_pct_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
country_spending_results = country_spend_pct_query_job.to_dataframe()

# View top few rows of results
print(country_spending_results.head())

            country_name  avg_ed_spending_pct
0                   Cuba            12.837270
1  Micronesia, Fed. Sts.            12.467750
2        Solomon Islands            10.001080
3                Moldova             8.372153
4                Namibia             8.349610


**2) Identify interesting codes to explore**

There are 1000s of codes in the dataset, so it would be time consuming to review them all. But many codes are available for only a few countries. When browsing the options for different codes, WE might restrict ourself to codes that are reported by many countries.

Write a query below that selects the indicator code and indicator name for all codes with at least 175 rows in the year 2016.

Requirements:

- You should have one row for each indicator code.
- The columns in your results should be called indicator_code, indicator_name, and num_rows.
- Only select codes with 175 or more rows in the raw database (exactly 175 rows would be included).
- To get both the indicator_code and indicator_name in your resulting DataFrame, you need to include both in your SELECT statement (in addition to a COUNT() aggregation). This requires you to include both in your GROUP BY clause.
- Order from results most frequent to least frequent.

In [10]:
import pandas as pd

code_count_query = """
                    SELECT indicator_code, indicator_name, COUNT(country_name) as num_rows
                    FROM `bigquery-public-data.world_bank_intl_education.international_education`
                    WHERE year = 2016
                    GROUP BY indicator_code, indicator_name
                    HAVING COUNT(1) >= 175
                    ORDER BY num_rows DESC
                    """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(code_count_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
code_count_results = code_count_query_job.to_dataframe()

# View top few rows of results
print(code_count_results.head())

      indicator_code                   indicator_name  num_rows
0        SP.POP.TOTL                Population, total       232
1        SP.POP.GROW     Population growth (annual %)       232
2     IT.NET.USER.P2  Internet users (per 100 people)       223
3     SP.POP.1564.TO    Population, ages 15-64, total       213
4  SP.POP.TOTL.FE.IN               Population, female       213
